# Выбор локации для скважины

Допустим, мы работаем в нефтедобывающей компании. Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Цель исследования:**    
* Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски.

**Ход иследования:**  
* Загрузка и подготовка данных
* Разбивка данных на выборки для обучения моделей
* Обучение моделей и получение предсказаний для трёх регионов
* Расчёт RMSE моделей
* Расчёт среднего запаса пресказанного сырья
* Расчёт достаточного объёма сырья для безубыточной разработки новой скважины
* Расчёт прибыли 200 лучших скважин в регионах
* Вычисление средней прибыли, 95%-го доверительного интервала и риска убытков

In [26]:
#Импортируем библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from numpy import sqrt

state = np.random.RandomState(2022)

## Загрузка и подготовка данных

In [2]:
#Сохраним данные в переменные
zero_data = pd.read_csv('geo_data_0.csv')
one_data = pd.read_csv('geo_data_1.csv')
two_data = pd.read_csv('geo_data_2.csv')

In [3]:
#Проверим наличие пропусков и типы данных
zero_data.info()
one_data.info()
two_data.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 100000 entries, 0 to 99999

Data columns (total 5 columns):

 #   Column   Non-Null Count   Dtype  

---  ------   --------------   -----  

 0   id       100000 non-null  object 

 1   f0       100000 non-null  float64

 2   f1       100000 non-null  float64

 3   f2       100000 non-null  float64

 4   product  100000 non-null  float64

dtypes: float64(4), object(1)

memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 100000 entries, 0 to 99999

Data columns (total 5 columns):

 #   Column   Non-Null Count   Dtype  

---  ------   --------------   -----  

 0   id       100000 non-null  object 

 1   f0       100000 non-null  float64

 2   f1       100000 non-null  float64

 3   f2       100000 non-null  float64

 4   product  100000 non-null  float64

dtypes: float64(4), object(1)

memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 100000 entries, 0 to 99999

Data columns (total 5 columns

In [4]:
#Выведем первые пять строк датафреймов на экран
display(zero_data.head())
display(one_data.head())
display(two_data.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [5]:
#Проверим датафреймы на дубликаты
display(zero_data.duplicated().sum())
display(one_data.duplicated().sum())
display(two_data.duplicated().sum())

0

0

0

In [6]:
#Проверим корреляцию признаков
display(zero_data.corr())
display(one_data.corr())
display(two_data.corr())

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


**Вывод:** 
* Данные сохранены в переменные, изучены
* Пропусков и дубликатов в данных нет
* В нулевом датафрейме присутствует небольшая отрицательная корреляция между признаками f0 и f1, а также небольшая положительная корреляция между f2 и product
* В первом датафрейме присутствует очень ольшая положительная корреляция между признаками f2 и product
* Во втором датафрейме также присутствует небольшая корреляция между признаками f2 и product

## Обучение и проверка модели

In [7]:
#Определим признаки и целевой признак для каждого датафрейма
zero_features = zero_data.drop(['id', 'product'], axis=1)
zero_target = zero_data['product']

one_features = one_data.drop(['id', 'product'], axis=1)
one_target = one_data['product']

two_features = two_data.drop(['id', 'product'], axis=1)
two_target = two_data['product']


In [8]:
#Разделим данные на выборки

zero_features_train, zero_features_valid, zero_target_train, zero_target_valid = train_test_split(zero_features,
                                                                                  zero_target,
                                                                                  test_size=0.25,
                                                                                 random_state=state,
                                                                                  shuffle=False)
one_features_train, one_features_valid, one_target_train, one_target_valid = train_test_split(one_features,
                                                                                  one_target,
                                                                                  test_size=0.25,
                                                                                 random_state=state,
                                                                                  shuffle=False)
two_features_train, two_features_valid, two_target_train, two_target_valid = train_test_split(two_features,
                                                                                  two_target,
                                                                                  test_size=0.25,
                                                                                 random_state=state,
                                                                                  shuffle=False)

In [9]:
#Проверим размер выборок
print(zero_features_train.shape)
print(one_features_train.shape)
print(two_features_train.shape)

(75000, 3)

(75000, 3)

(75000, 3)


In [10]:
#Обучим модель и сделаем предсказание для нулевого региона
scaler = StandardScaler()
scaler.fit(zero_features_train)

zero_features_train_scal = scaler.transform(zero_features_train)
zero_features_valid_scal = scaler.transform(zero_features_valid)

model = LinearRegression()
model.fit(zero_features_train_scal, zero_target_train)

zero_predictions = model.predict(zero_features_valid_scal)

#Приведём предсказания к Series и установим индексы, аналогичные zero_target_valid
zero_np_array = np.array(zero_predictions)
zero_pred = pd.Series(zero_np_array, index=zero_target_valid.index)

In [11]:
#Обучим модель и сделаем предсказание для первого региона
scaler = StandardScaler()
scaler.fit(zero_features_train)

one_features_train_scal = scaler.transform(one_features_train)
one_features_valid_scal = scaler.transform(one_features_valid)

model = LinearRegression()
model.fit(one_features_train_scal, one_target_train)

one_predictions = model.predict(one_features_valid_scal)

#Приведём предсказания к Series и установим индексы, аналогичные one_target_valid
one_np_array = np.array(one_predictions)
one_pred = pd.Series(one_np_array, index=one_target_valid.index)

In [12]:
#Обучим модель и сделаем предсказание для второго региона
scaler = StandardScaler()
scaler.fit(zero_features_train)

two_features_train_scal = scaler.transform(two_features_train)
two_features_valid_scal = scaler.transform(two_features_valid)

model = LinearRegression()
model.fit(two_features_train_scal, two_target_train)

two_predictions = model.predict(two_features_valid_scal)

#Приведём предсказания к Series и установим индексы, аналогичные two_target_valid
two_np_array = np.array(two_predictions)
two_pred = pd.Series(two_np_array, index=two_target_valid.index)

In [13]:
#Сохраним правильные ответы и выведем их на экран
zero_mean = zero_target_valid.mean()
one_mean = one_target_valid.mean()
two_mean = two_target_valid.mean()
print(zero_mean)
print(one_mean)
print(two_mean)

92.42663467014862

68.94008904238618

94.91236791621232


In [14]:
#Рассчитаем RMSE моделей
zero_rmse = mean_squared_error(zero_target_valid, zero_pred) ** 0.5
one_rmse = mean_squared_error(one_target_valid, one_pred) ** 0.5
two_rmse = mean_squared_error(two_target_valid, two_pred) ** 0.5

In [15]:
#Рассчитаем средний запас предсказанного сырья
zero_mean = zero_pred.mean()
one_mean = one_pred.mean()
two_mean = two_pred.mean()

In [16]:
#Выведем на экран RMSE и средний запас предсказанного сырья каждой модели
print('RMSE нулевого региона:', zero_rmse)
print('Средний запас нулевого региона:', zero_mean)
print('RMSE первого региона:', one_rmse)
print('Средний запас первого региона:', one_mean)
print('RMSE второго региона:', two_rmse)
print('Средний запас второго региона:', two_mean)

RMSE нулевого региона: 37.650563264123434

Средний запас нулевого региона: 92.59368096991008

RMSE первого региона: 0.8944456516942326

Средний запас первого региона: 68.95753362359754

RMSE второго региона: 40.00368126890319

Средний запас второго региона: 95.27546163252782


**Вывод:**  
* По среднему предсказанному запасу перспективнее всего нулевой и второй регионы
* RMSE ниже всего у первого региона

## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [17]:
#Сохраним значения в переменных
budget = 10000000000
barel_profit = 450000
max_points = 500
best_max_points = 200

In [18]:
#Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины
print('Объём сырья для безубыточной разработки скважины:', budget/barel_profit/best_max_points)

Объём сырья для безубыточной разработки скважины: 111.11111111111111


In [19]:
#Сравним полученный объём сырья со средним запасом в каждом регионе
print('Средний запас в нулевом регионе', zero_data['product'].mean())
print('Средний запас в первом регионе', one_data['product'].mean())
print('Средний запас во втором регионе', two_data['product'].mean())

Средний запас в нулевом регионе 92.50000000000001

Средний запас в первом регионе 68.82500000000002

Средний запас во втором регионе 95.00000000000004


**Вывод:**  
Ни в одном из регионов средний запас не достигает нужной отметки в 111.11, однако наша модель прогнозирует хорошие запасы скважин в первом регионе, несмотря на довольно маленькое среднее.

## Расчёт прибыли и рисков 

In [20]:
#Напишем функцию расчёта прибыли
def revenue(target_valid, probabilities, count):
    prob_sorted = pd.Series(probabilities).sort_values(ascending=False)
    selected = target_valid[prob_sorted.index][:count]
    goal_sum = selected.sum()
    return round((goal_sum * barel_profit) - budget, 2)

In [21]:
#Рассчитаем прибыль
print('Прибыль 200 лучших скважин в нулевом регионе:', revenue(zero_target_valid, zero_pred, best_max_points))
print('Прибыль 200 лучших скважин в первом регионе:', revenue(one_target_valid, one_pred, best_max_points))
print('Прибыль 200 лучших скважин во втором регионе:', revenue(two_target_valid, two_pred, best_max_points))

Прибыль 200 лучших скважин в нулевом регионе: 3356865603.95

Прибыль 200 лучших скважин в первом регионе: 2415086696.68

Прибыль 200 лучших скважин во втором регионе: 2588651491.44


In [22]:
#Напишем функцию bootstrap
def bootstr(target_valid, predictions):

    values = []
    for i in range(1000):
        target_sample = target_valid.sample(replace = True, random_state = state, n=max_points)
        predictions_sample = predictions[target_sample.index]

        values.append(revenue(target_valid, predictions_sample, best_max_points))

    values = pd.Series(values)
    return values 

In [23]:
#Найдём распределение прибыли для каждого региона
zero_rasp = bootstr(zero_target_valid, zero_pred)
one_rasp = bootstr(one_target_valid, one_pred)
two_rasp = bootstr(two_target_valid, two_pred)

In [27]:
#Напишем функции для вычисления средней прибыли, 95%-го доверительного интервала и риска убытков
def mean_revenue(values):
    print("Средняя выручка:", values.mean())
    
def dov_int(values):
    print("95%-й доверительный интервал:", values.quantile(0.025), '-', values.quantile(0.975))
    
def risks(values):
    print("Риск убытков:", st.percentileofscore(values, 0), '%')

In [28]:
mean_revenue(zero_rasp)
dov_int(zero_rasp)
risks(zero_rasp)

Средняя выручка: 417981062.47498

95%-й доверительный интервал: -100642480.11125 - 940737222.4679997

Риск убытков: 4.800000000000001 %


In [29]:
mean_revenue(one_rasp)
dov_int(one_rasp)
risks(one_rasp)

Средняя выручка: 466833977.84547997

95%-й доверительный интервал: 69515438.6515 - 837478118.5944997

Риск убытков: 1.0 %


In [30]:
mean_revenue(two_rasp)
dov_int(two_rasp)
risks(two_rasp)

Средняя выручка: 384069231.80391

95%-й доверительный интервал: -116373605.90649997 - 900804348.1377499

Риск убытков: 6.9 %


**Вывод:**  
Для разработки скважин подходит первый регион, т.к. риск убытков у него составляет 1%, что меньше, чем у двух других регионов, а средняя выручка, наоборот, выше, чем у остальных регионов.

Общий вывод
====  
  
Для исследования было сделано следующее:  
* Данные загружены и подготовлены к дальнейшей работе
* Обучены и проверены модели
* Подсчитаны средний запас сырья и RMSE модели
* Рассчитан достаточный объём сырья для безубыточной разработки новой скважины
* Рассчитана прибыль для скважин с максимальным оъёмом сырья
* Подсчитаны: средняя прибыль, 95%-й доверительный интервал и риск убытков 
  
По результатам исследования лучшим регионом для разработки скважин был выбран первый регион, т.к., по сравнению с двумя другими регионами, у первого наименьшее значение риска убытков и высокая средняя выручка.